# Worksheet 08

Name:  Anulika Nnadi
UID: U14553525

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.428208232879302, 4.225824553935645, 5.803500001720045, 4.350957020656254, 5.627135344352418, 4.530722540358561, 6.79036339691899, 4.140926216277345, 4.912049732049778, 4.013063003698743]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
c2 = np.random.normal( 8 , 1, 10 ).tolist()
print(c2)

[8.733917573715408, 8.641611843101312, 8.152201361230931, 7.3312570508407156, 7.1900024661486714, 7.332578753458971, 6.551265565567644, 6.606344913583632, 7.937200640504548, 8.210191688061036]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[4.013063003698743, 4.912049732049778, 8.210191688061036, 7.937200640504548, 6.606344913583632, 4.140926216277345, 6.551265565567644, 6.79036339691899, 7.332578753458971, 4.530722540358561]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

for each individual probability's mean variance and a probabilty of x being i-th species. Guassian Mixture Model when all are normal distribution with parameters -> start with random computer for all Xi by using compute and update from repeat 2 & 3 until convergence

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] != 0, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)  ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

/Users/aninja/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[4.013063003698743, 4.912049732049778, 4.140926216277345, 4.530722540358561]
[8.210191688061036, 7.937200640504548, 6.606344913583632, 6.551265565567644, 6.79036339691899, 7.332578753458971]
P(C_1) = 0.4,  P(C_2) = 0.6
mean_1 = 4.399190373096107,  mean_2 = 7.237990826349136
var_1 = 0.12403003829654069,  var_2 = 8.477780382417366


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [7]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )
    
probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  4.013063003698743
probability of observing that point if it came from cluster 0 =  0.02528256594821512
probability of observing that point if it came from cluster 1 =  0.04377298680823249
point =  4.912049732049778
probability of observing that point if it came from cluster 0 =  0.0006231907770771493
probability of observing that point if it came from cluster 1 =  0.04531926003324048
point =  8.210191688061036
probability of observing that point if it came from cluster 0 =  3.130625243491937e-205
probability of observing that point if it came from cluster 1 =  0.04674899441319917
point =  7.937200640504548
probability of observing that point if it came from cluster 0 =  6.52574357552598e-177
probability of observing that point if it came from cluster 1 =  0.046897621849497396
point =  6.606344913583632
probability of observing that point if it came from cluster 0 =  5.528472772442548e-69
probability of observing that point if it came from cluster 1 =  0.046926967114670974
poin

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [ ]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), ... ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), ... ]
var = [ ... , ... ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x )
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

point =  4.013063003698743
probability of observing that point if it came from cluster 0 =  0.02528256594821512
probability of observing that point if it came from cluster 1 =  0.04377298680823249
point =  4.912049732049778
probability of observing that point if it came from cluster 0 =  0.0006231907770771493
probability of observing that point if it came from cluster 1 =  0.04531926003324048
point =  8.210191688061036
probability of observing that point if it came from cluster 0 =  3.130625243491937e-205
probability of observing that point if it came from cluster 1 =  0.04674899441319917
point =  7.937200640504548
probability of observing that point if it came from cluster 0 =  6.52574357552598e-177
probability of observing that point if it came from cluster 1 =  0.046897621849497396
point =  6.606344913583632
probability of observing that point if it came from cluster 0 =  5.528472772442548e-69
probability of observing that point if it came from cluster 1 =  0.046926967114670974
poin

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
assignments = [[], []]
for x in zip(data, prob_c0_x):
    p = x[1]
    d = x[0]
    if random.random() <= p:
        assignments[0].append(d)
    else:
        assignments[1].append(d)
print(f"points in cluster 0 : {assignments[0]}\npoints in cluster 1 : {assignments[1]}")

points in cluster 0 : [4.013063003698743, 4.140926216277345, 4.530722540358561]
points in cluster 1 : [4.912049732049778, 8.210191688061036, 7.937200640504548, 6.606344913583632, 6.551265565567644, 6.79036339691899, 7.332578753458971]
